In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
os.chdir("/home/zhzhou/GNN_E")
import numpy as np
import pandas as pd
import torch
from torch_geometric.loader import DataLoader
from math import ceil
from utils_models import *
from scripts.models import *                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
from GCN.GCN_models import *

/home/zhzhou/.conda/envs/GNN_E_clone/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#data_path = 'data/processed/all_movies_labelled_13_single.csv'
data_path = '/home/dalai/GNN_E/data/processed/all_movies_labelled_13_single_balanced.csv'
df =  pd.read_csv(data_path, index_col=0)

In [10]:
df_single = df

In [104]:

import numpy as np
from scipy.stats import mode
from collections import Counter
def majority_vote(x):
    try:
        m = mode(x)
        return m.mode if m.count > 0 else -1
    except (IndexError, TypeError):
        return -1  # Default value when an error occurs

def second_highest_vote(x):
    majority = majority_vote(x)
    if majority == -1:
        # Count frequencies of all values
        counts = Counter(x)
        # Remove the majority vote (-1) from the count
        del counts[majority]
        # Get the second-highest vote (if any)
        if counts:
            second_highest = max(counts, key=counts.get)
            return second_highest
        else:
            # If no second-highest vote, return -1
            return -1
    else:
        return majority


super_subject = (
    df.groupby(["vindex", "movie", "timestamp_tr"], as_index=False)
    .agg({
        "score": "mean",
        "label": lambda x: ", ".join(map(str, x))  # Use the helper function
    })
)

In [105]:
super_subject['label'] = super_subject['label'].apply( lambda x: list(map(int, x.split(", "))))

In [106]:
super_subject['label_'] =  super_subject['label'].apply(lambda x: second_highest_vote(x))

In [119]:
df_single = super_subject
df_single['id'] = 0
df_single['label'] = df_single['label_']

In [11]:
df_single

,Unnamed: 0,vindex,score,id,movie,timestamp_tr,label
0,0,0,-21.180140,1,0,0,-1
1,1,0,-14.295689,1,0,1,-1
2,2,0,-5.882564,1,0,2,-1
3,3,0,30.127113,1,0,3,-1
4,4,0,41.830845,1,0,4,-1
...,...,...,...,...,...,...,...
118876369,118876369,413,-21.945122,32,13,763,-1
118876370,118876370,413,3.040432,32,13,764,-1
118876371,118876371,413,-36.525375,32,13,765,-1
118876372,118876372,413,-7.233601,32,13,766,-1


In [12]:
os.getcwd()

'/home/zhzhou/GNN_E'

In [13]:
with open("/home/dalai/GNN_E/data/raw/labels/run_onsets.pkl", "rb") as file:
    delta_time = pkl.load(file)

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
df_train, df_test = split_train_test_vertically(
        df_single
    )
#df_train,_, df_test = split_train_val_test_horizontally(
#       df_single, 
#        percentage_val=0, #0 to not have nay val set
#        path_pickle_delay="/home/dalai/GNN_E/data/raw/labels/run_onsets.pkl",
#        path_movie_title_mapping="data/raw/labels/category_mapping_movies.csv", 
#        tr_len=1.3
#    )
dataset_train = DatasetEmo_fast(df_train, device=DEVICE,FN = 'Limbic',initial_adj_method='clique_edgeAttr_FC_window', node_feat="symmetricwindow")
dataset_test = DatasetEmo_fast(df_test, device = DEVICE,FN = 'Limbic',initial_adj_method='clique_edgeAttr_FC_window', node_feat="symmetricwindow")

graphs_list_train = dataset_train.get_graphs_list()
graphs_list_test = dataset_test.get_graphs_list()


Movies in this df: [ 0  1  2  3  4  5  6  8 11 12 13]


 13%|█▎        | 4/30 [01:08<07:22, 17.02s/it]


KeyboardInterrupt: 

In [ ]:
from torch_geometric.loader import DataLoader
from math import ceil
batch_size = 16
train_loader = DataLoader(graphs_list_train, batch_size=batch_size, num_workers=4, persistent_workers=True)
test_loader = DataLoader(graphs_list_test, batch_size=batch_size, num_workers=4, persistent_workers=True)

num_batches_train = ceil(len(graphs_list_train) / batch_size)
num_batches_test = ceil(len(graphs_list_test) / batch_size)

print(f"There are {len(graphs_list_train)} graphs in the train set.")
print(f"There are {len(graphs_list_test)} graphs in the test set.")
print(f"N batches in train: {num_batches_train}")
print(f"N batches in test: {num_batches_test}")

There are 2787 graphs in the train set.
There are 931 graphs in the test set.
N batches in train: 175
N batches in test: 59


In [ ]:
from tqdm import tqdm
input_dim = 9 
hidden_dim1 = 128
hidden_dim2 = 64
output_dim = 13  # Number of classes
model = SimpleGCN(input_dim, hidden_dim1, hidden_dim2, output_dim).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, capturable=False)
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
num_epochs = 1
# Training loop
#torch.cuda.empty_cache()
model, results_dict = GCN_train(model=model, optimizer=optimizer, loss_fn=loss_fn, 
                                    train_loader=train_loader, test_loader=test_loader, device=DEVICE, num_epochs=50)

with open('data/results/GCN/GCNModel_result_all.pkl','wb') as f:
    pkl.dump(results_dict,f)

with open('data/results/GCN/GCNModel_model_all.pkl','wb') as f:
    pkl.dump(model.state_dict(), f)

	Epoch 1/50, Loss: 3.516248039518084, Train accuracy: 0.10010764262648009



	Epoch 1/50, Loss: 2.7774, test accuracy: 0.0698

Result saved!


	Epoch 2/50, Loss: 2.650294736453465, Train accuracy: 0.06458557588805167



	Epoch 2/50, Loss: 2.6796, test accuracy: 0.0559

Result saved!


	Epoch 3/50, Loss: 2.594392454964774, Train accuracy: 0.06350914962325081



...

	Epoch 39/50, Loss: 2.496046209335327, Train accuracy: 0.10190168640114819



	Epoch 39/50, Loss: 3.1577, test accuracy: 0.0462

Result saved!


	Epoch 40/50, Loss: 2.491653105872018, Train accuracy: 0.10764262648008611



	Epoch 40/50, Loss: 3.2005, test accuracy: 0.0494

Result saved!


KeyboardInterrupt: 

all_labels = torch.cat(results_dict['best_prediction']).cpu().numpy()

# Calculate frequency of each unique label
unique_labels, counts = torch.unique(torch.tensor(all_labels), return_counts=True)

# Plot the frequency distribution
plt.bar(unique_labels.numpy(), counts.numpy())
plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.title('Frequency of Different Labels')
plt.show()

real_y = []
for i in graphs_list_test:
    real_y.append(i.y.item())

unique_labels, counts = torch.unique(torch.tensor(real_y), return_counts=True)

# Plot the frequency distribution
plt.bar(unique_labels.numpy(), counts.numpy())
plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.title('Frequency of Different Labels')
plt.show()

import matplotlib.pyplot as plt
predicted_label = []
for i in results_dict['best_prediction']:
    predicted_label = list(i)
predicted_label = [i.item() for i in predicted_label]

file_name = 'data/results/GCN/GCNModel.pth'
torch.save(model.state_dict(), file_name)

def test():
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct = 0  # To keep track of the number of correct predictions
    total = 0    # To keep track of the total number of samples

    # No need for gradients during testing
    with torch.no_grad():
        progress_bar = tqdm(loader_test, desc="Testing", leave=False)
        for i, batch in enumerate(progress_bar):
            batch = batch.to(DEVICE)

            # Forward pass
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = loss_fn(out, batch.y)

            # Update total loss
            total_loss += loss.item()

            # Get the predicted class (index of the max logit)
            _, predicted = torch.max(out, dim=1)

            # Update the number of correct predictions and total samples
            correct += (predicted == batch.y).sum().item()
            print(predicted)
            print(batch.y)
            total += batch.y.size(0)

            # Update the progress bar with loss
            progress_bar.set_postfix({"Test Loss": predicted})

    # Calculate the average loss and accuracy
    avg_loss = total_loss / len(loader_test)
    accuracy = correct / total * 100  # Accuracy as a percentage

    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")

    return avg_loss, accuracy

test()

In [ ]:
with open('data/results/GCN/GCNModel_result_all.pkl','wb') as f:
    pkl.dump(results_dict,f)